In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(200, 16)

In [3]:
len(df[df.tiling_compatible == True]) 

33

In [4]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format     87
1         group_structure     39
2                    None     33
3        no_xy_dimensions     21
4                 timeout     12
5  tile_generation_failed      4
6  cant_extract_variables      2
7       failed_to_extract      2


In [15]:
list(df[df.incompatible_reason == 'failed_to_extract'].error_message)

['Forbidden',
 "did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'argo', 'cfgrib', 'gini', 'kerchunk', 'pydap', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:\nhttps://docs.xarray.dev/en/stable/getting-started-guide/installing.html\nhttps://docs.xarray.dev/en/stable/user-guide/io.html"]

In [8]:
list(df[df.incompatible_reason == 'cant_open_file'].collection_concept_id)

['C1615905582-GES_DISC',
 'C1288937556-GES_DISC',
 'C2368310204-GES_DISC',
 'C1276812830-GES_DISC',
 'C1276812838-GES_DISC',
 'C1276812851-GES_DISC',
 'C1575363595-GES_DISC',
 'C2723754864-GES_DISC',
 'C2368308287-GES_DISC',
 'C3294057315-ASF',
 'C2089270961-GES_DISC',
 'C1276812884-GES_DISC',
 'C1276812759-GES_DISC',
 'C2565093311-NSIDC_CPRD',
 'C2912085112-GES_DISC',
 'C2033151148-GES_DISC',
 'C2734202914-LPCLOUD',
 'C1700900796-GES_DISC']

In [33]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

27                                               [SWIR1]
29                                               [TIRS2]
34     [latitude, longitude, scan_datetime, year, mon...
37     [aerosol_optical_thickness_550_land, aerosol_o...
43     [Aerosol_Optical_Thickness_550_Expected_Uncert...
                             ...                        
623    [Delta_Time, SZA, VZA, SAz, VAz, Longitude, La...
625                                             [weight]
640    [obs_id, fov_obs_id, obs_time_tai93, obs_time_...
643    [EFLUXICE, EFLUXWTR, FRSEAICE, HFLUXICE, HFLUX...
646                    [OriginalInputFiles, XmlMetadata]
Name: data_variables, Length: 160, dtype: object

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
